# Play ground for Engeenering the Prompt

In [1]:
import os
import openai
import pandas as pd
import query_data
import datetime as dt
import pytz
from langchain.llms import OpenAIChat
import configparser
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
config = configparser.ConfigParser()
config.read('config.ini')
openai.api_key = config['openai']['api_key']
os.environ["OPENAI_API_KEY"] = config['openai']['api_key']


In [2]:
llm = OpenAIChat(temperature=0)

## Simple summary

In [3]:

prompt_narrative_crypto = """Create a list of crypto $TOKEN from the text and follow this guide:
                             1) Tokens mentioned in the tweets should be in the format $TOKEN, but not necessarily.
                             2) Unify information for each $TOKEN.
                             3) For each $TOKEN, inclued narrative  or relevant information.
                             4) Add catalysts for each token if possible.
                             5) if No specific $TOKEN mentioned, join all the text together.
    response list example: "1) $BTC:  Posible pump due to Elon Musk's tweet. Catalyst: Next convention in march.
                       2) $HPK:  Strong narrative  Catalyst: No info.
                       3) General Narrative:
                          - Many altcoins are currently experiencing a significant drop in price.
                          - CPI data is expected to be released today.
                       "

   {text}
   """


In [6]:
prompt_narrative_crypto = """Given the following text:"{text}"

Create a list of crypto tokens with the corresponding narratives
and potential catalysts for each token. Use the following guidelines:

Use the format $TOKEN or TOKEN is used when mentioning a specific token in the tweets.
If no specific $TOKEN is mentioned, summarize the general narrative.
Unify information for each $TOKEN.
Include relevant information for each $TOKEN, such as market trends, news, or events.
Add potential catalysts for each token if possible.

After creating the list, structure your response as follows:

"1) $TOKEN1: [narrative]. Catalysts: [potential catalysts].
2) $TOKEN2: [narrative]. Catalysts: [potential catalysts].
.....
n) $TOKENn: [narrative]. Catalysts: [potential catalysts].
General Narrative: [general summary of market trends]."

"""

In [7]:
# load tweets
tweets = pd.read_pickle("tweets_crypto.pkl")
# get tweets from last 24 hours
last_24 = dt.datetime.now(pytz.UTC) - dt.timedelta(hours=24)
tweets = [ tweet for tweet in tweets  if tweet.created_at >= last_24]
docs = query_data.create_thread_docs(tweets, filer_len=2)

In [8]:

PROMPT = PromptTemplate(template=prompt_narrative_crypto, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
chain.run(docs)

'\n\n1) $STG: Bought a little. Will add on further strength in the v low 0.80’s. Catalysts: potential breakout on a break of 0.17, targets at 0.22, 0.25, and 0.30.\n2) $LAMB: AI-focused token with 50% growth and 1-day pump duration. Catalysts: AI narrative.\n3) $CULT: zkEVM-focused token with 30% growth and 1-day pump duration. Catalysts: zkEVM narrative.\n4) $AIPAD: Arbitrum-focused token with 20% growth and 2-hour pump duration. Catalysts: Arbitrum narrative.\nGeneral Narrative: Many alts are currently 30-40% off from highs just 2 weeks ago. Binance is dropping the ball on offering spot twap. Staked ETH participants are at a loss with the average staking price at $2.1k.'

## Combine summary win tokens more than 4000

In [48]:
refine_template = (
        "your job is to create a list of crypto $TOKEN from the text and follow this guide:\n"
                             "1) Tokens mentioned in the tweets should be in the format $TOKEN\n"
                             "2) Unify information for each $TOKEN.\n"
                             "3) For each $TOKEN, inclued narrative  or relevant information.\n"
                             "4) Add catalysts for each token if possible.\n"
                             "5) if No specific $TOKEN mentioned, include a general narrative.\n"
        "We have provided an existing respond up to a certain point: {existing_answer}\n"
        "We have the opportunity to refine the existing list of $TOKEN \n"
        "(only if needed) with some more text below.\n"
        "------------\n"
        "{text}\n"
        "------------\n"
        "Given the new text, refine the original list"
        "If the text isn't useful, return the original list."
    )
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True,
                             question_prompt=PROMPT, refine_prompt=refine_prompt)
chain({"input_documents": docs}, return_only_outputs=True)['output_text']
